## Установка специфичной версии open3d

In [1]:
!pip install -U -f https://www.open3d.org/docs/latest/getting_started.html --only-binary open3d open3d --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 97.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 13.7 MB/s eta 0:00:00


## Импорты

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torchvision.models import ResNet50_Weights
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import open3d as o3d
from PIL import Image
import torchvision.transforms as transforms
import cv2
from tqdm import tqdm
import time
import multiprocessing
import matplotlib.pyplot as plt
import math
from pathlib import Path
import json
from torch.optim.lr_scheduler import CosineAnnealingLR

## Класс для работы с датасетом и предобработки изображений и 3d моделей

In [ ]:
class VoxelDataset(Dataset):
    """Dataset for image to voxel reconstruction"""
    def __init__(self, root_dir, transform=None, image_size=224, voxel_size=32, voxel_threshold=0.5):
        self.root_dir = root_dir
        self.image_size = image_size
        self.voxel_size = voxel_size
        self.voxel_threshold = voxel_threshold
        
        self.transform = transform if transform is not None else transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        self.images_dir = os.path.join(root_dir, 'images')
        self.mesh_dir = os.path.join(root_dir, 'stl')
        self.image_files = []
        for f in os.listdir(self.images_dir):
            if f.endswith(('.png', '.jpg', '.jpeg')):
                mesh_file = os.path.join(self.mesh_dir, f.rsplit('.', 1)[0] + '.stl')
                if os.path.exists(mesh_file):
                    self.image_files.append(f)
        
        if not self.image_files:
            raise RuntimeError(f"No valid image-mesh pairs found in {self.images_dir}")
        
        print("Preloading voxel grids...")
        self.voxel_grids = {}
        with tqdm(total=len(self.image_files), desc="Loading voxel grids", 
                 bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]',
                 ncols=100, leave=True) as pbar:
            for f in self.image_files:
                mesh_name = os.path.join(self.mesh_dir, f.rsplit('.', 1)[0] + '.stl')
                voxels = self._preprocess_voxel_grid(mesh_name)
                self.voxel_grids[f] = voxels
                pbar.update(1)
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = os.path.join(self.images_dir, self.image_files[idx])
        image = self._preprocess_image(img_name)
        voxels = self.voxel_grids[self.image_files[idx]]
        
        return {
            'image': image,
            'voxels': voxels.unsqueeze(0),
            'filename': self.image_files[idx]
        }
    
    def _preprocess_voxel_grid(self, mesh_path):
        try:
            mesh = o3d.io.read_triangle_mesh(mesh_path)
            if not mesh.has_vertices():
                raise ValueError(f"Empty mesh loaded from {mesh_path}")
            
            mesh.translate(-mesh.get_center())
            vertices = np.asarray(mesh.vertices)
            scale = np.max(np.abs(vertices))
            if scale > 0:
                mesh.scale(1.0 / scale, center=mesh.get_center())
            
            voxel_size = 2.0 / self.voxel_size  # Scale to fit in [-1, 1]³
            voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(
                mesh,
                voxel_size=voxel_size
            )
            
            voxels = np.zeros((self.voxel_size, self.voxel_size, self.voxel_size), dtype=np.float32)
            voxel_centers = np.asarray([voxel.grid_index for voxel in voxel_grid.get_voxels()])
            
            valid_indices = np.all((voxel_centers >= 0) & (voxel_centers < self.voxel_size), axis=1)
            voxel_centers = voxel_centers[valid_indices]
            
            voxels[voxel_centers[:, 0], voxel_centers[:, 1], voxel_centers[:, 2]] = 1.0
            return torch.FloatTensor(voxels)
            
        except Exception as e:
            raise RuntimeError(f"Error preprocessing mesh file {mesh_path}: {str(e)}")
    
    def _preprocess_image(self, img_path, visualize=False):
        try:
            image = Image.open(img_path).convert('RGB')
            image_np = np.array(image)
            gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
            clahe = cv2.createCLAHE(clipLimit=3.4, tileGridSize=(4,4))
            enhanced = clahe.apply(gray)
            gamma = 1.5
            lookUpTable = np.empty((1,256), np.uint8)
            for i in range(256):
                lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
            enhanced = cv2.LUT(enhanced, lookUpTable)
            filtered = cv2.bilateralFilter(enhanced, 3, 15, 15)
            edges = cv2.Canny(filtered, threshold1=6, threshold2=26)
            kernel = np.ones((3,3), np.uint8)
            edges = cv2.dilate(edges, kernel, iterations=2)
            edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
            edges_rgb[edges > 0] = [255, 255, 255]  # Set edges to pure white
            combined = cv2.addWeighted(image_np, 0.5, edges_rgb, 0.5, 0)
            if visualize:
                cv2.imshow('Edge Detection Result', combined)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
            combined_image = Image.fromarray(combined)
            image = self.transform(combined_image)
            return image
        except Exception as e:
            raise RuntimeError(f"Error preprocessing image {img_path}: {str(e)}")

## Модель

In [ ]:

class EncoderDecoder3D(nn.Module):
    def __init__(self, voxel_size=32):
        super(EncoderDecoder3D, self).__init__()
        
        # Encoder (using ResNet50)
        weights = ResNet50_Weights.IMAGENET1K_V1
        resnet = models.resnet50(weights=weights)
        self.encoder = nn.Sequential(*list(resnet.children())[:-2])
        
        self.encoder_output_size = 2048 * 7 * 7
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(self.encoder_output_size, 4 * 4 * 4 * 64),
            nn.BatchNorm1d(4 * 4 * 4 * 64),
            nn.ReLU(),
            
            nn.ConvTranspose3d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            
            nn.ConvTranspose3d(32, 16, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            
            nn.ConvTranspose3d(16, 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(8),
            nn.ReLU(),
            
            nn.Conv3d(8, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )
        self.use_checkpointing = True
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d) or isinstance(m, nn.ConvTranspose3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm3d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        if self.use_checkpointing and self.training:
            features = torch.utils.checkpoint.checkpoint(self.encoder, x)
        else:
            features = self.encoder(x)
        
        features = features.view(features.size(0), -1)
        
        x = self.decoder[0:3](features)
        x = x.view(-1, 64, 4, 4, 4)
        x = self.decoder[3:](x)
        
        return x


## Класс для тренировки модели

In [ ]:
class Trainer:
    def __init__(self, model, train_loader, val_loader, optimizer, scheduler, device, log_dir='runs'):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.max_grad_norm = 1.0
        self.accumulation_steps = 4
        
        self.train_losses = []
        self.val_losses = []
        self.train_ious = []
        self.val_ious = []
        self.train_dices = []
        self.val_dices = []
    
    def calculate_metrics(self, pred_voxels, target_voxels):
        pred_probs = torch.sigmoid(pred_voxels)
        pred_voxels = (pred_probs > 0.5).float()
        
        # IoU
        intersection = (pred_voxels * target_voxels).sum()
        union = pred_voxels.sum() + target_voxels.sum() - intersection
        iou = (intersection + 1e-6) / (union + 1e-6)
        
        # Dice
        dice = (2. * intersection + 1e-6) / (pred_voxels.sum() + target_voxels.sum() + 1e-6)
        
        return iou.item(), dice.item()
    
    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        total_iou = 0
        total_dice = 0
        self.optimizer.zero_grad()
        
        pbar = tqdm(total=len(self.train_loader), desc=f'Epoch {epoch}')
        for batch_idx, batch in enumerate(self.train_loader):
            images = batch['image'].to(self.device)
            target_voxels = batch['voxels'].to(self.device)
            
            pred_voxels = self.model(images)
            loss = F.binary_cross_entropy(pred_voxels, target_voxels)
            loss = loss / self.accumulation_steps
            
            loss.backward()
            
            if (batch_idx + 1) % self.accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.max_grad_norm)
                self.optimizer.step()
                self.optimizer.zero_grad()
            
            # Calculate metrics
            iou, dice = self.calculate_metrics(pred_voxels, target_voxels)
            
            total_loss += loss.item() * self.accumulation_steps
            total_iou += iou
            total_dice += dice
            
            pbar.update(1)
            pbar.set_postfix({
                'loss': f"{loss.item() * self.accumulation_steps:.4f}",
                'iou': f"{iou:.4f}",
                'dice': f"{dice:.4f}"
            })
            
            if batch_idx % 5 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        pbar.close()
        avg_loss = total_loss / len(self.train_loader)
        avg_iou = total_iou / len(self.train_loader)
        avg_dice = total_dice / len(self.train_loader)
        
        self.train_losses.append(avg_loss)
        self.train_ious.append(avg_iou)
        self.train_dices.append(avg_dice)
        
        return avg_loss, avg_iou, avg_dice
    
    def validate(self, epoch):
        self.model.eval()
        total_loss = 0
        total_iou = 0
        total_dice = 0
        
        pbar = tqdm(total=len(self.val_loader), desc=f'Validation {epoch}')
        
        with torch.no_grad():
            for batch_idx, batch in enumerate(self.val_loader):
                images = batch['image'].to(self.device)
                target_voxels = batch['voxels'].to(self.device)
                
                pred_voxels = self.model(images)
                loss = F.binary_cross_entropy(pred_voxels, target_voxels)
                
                # Calculate metrics
                iou, dice = self.calculate_metrics(pred_voxels, target_voxels)
                
                total_loss += loss.item()
                total_iou += iou
                total_dice += dice
                
                pbar.update(1)
                pbar.set_postfix({
                    'val_loss': f"{loss.item():.4f}",
                    'iou': f"{iou:.4f}",
                    'dice': f"{dice:.4f}"
                })
        
        pbar.close()
        avg_loss = total_loss / len(self.val_loader)
        avg_iou = total_iou / len(self.val_loader)
        avg_dice = total_dice / len(self.val_loader)
        
        self.val_losses.append(avg_loss)
        self.val_ious.append(avg_iou)
        self.val_dices.append(avg_dice)
        
        return avg_loss, avg_iou, avg_dice
    
    def plot_metrics(self):
        """Plot training and validation metrics"""
        epochs = range(1, len(self.train_losses) + 1)
        
        # Create figure with subplots
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15))
        
        # Plot losses
        ax1.plot(epochs, self.train_losses, 'b-', label='Train Loss')
        ax1.plot(epochs, self.val_losses, 'r-', label='Val Loss')
        ax1.set_title('Training and Validation Loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True)
        
        # Plot IoU
        ax2.plot(epochs, self.train_ious, 'b-', label='Train IoU')
        ax2.plot(epochs, self.val_ious, 'r-', label='Val IoU')
        ax2.set_title('Training and Validation IoU')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('IoU')
        ax2.legend()
        ax2.grid(True)
        
        # Plot Dice
        ax3.plot(epochs, self.train_dices, 'b-', label='Train Dice')
        ax3.plot(epochs, self.val_dices, 'r-', label='Val Dice')
        ax3.set_title('Training and Validation Dice Coefficient')
        ax3.set_xlabel('Epoch')
        ax3.set_ylabel('Dice Coefficient')
        ax3.legend()
        ax3.grid(True)
        
        plt.tight_layout()
        plt.savefig('training_metrics.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    def train(self, num_epochs):
        print(f"Starting training for {num_epochs} epochs...")
        print(f"Training on device: {self.device}")
        print(f"Number of training batches: {len(self.train_loader)}")
        print(f"Number of validation batches: {len(self.val_loader)}")
        
        for epoch in range(num_epochs):
            train_loss, train_iou, train_dice = self.train_epoch(epoch)
            val_loss, val_iou, val_dice = self.validate(epoch)
            
            self.scheduler.step()
            
            if (epoch + 1) % 10 == 0:
                self.save_predictions(epoch)
            
            print(f'\nEpoch {epoch}:')
            print(f'Train Loss: {train_loss:.4f}, IoU: {train_iou:.4f}, Dice: {train_dice:.4f}')
            print(f'Val Loss: {val_loss:.4f}, IoU: {val_iou:.4f}, Dice: {val_dice:.4f}')
        
        self.plot_metrics()
        self.generate_validation_predictions()
        
        print("Training completed!")

def evaluate_voxels(pred_voxels, target_voxels):
    pred_probs = torch.sigmoid(pred_voxels)
    pred_voxels = (pred_probs > 0.5).float()
    
    # Compute IoU
    intersection = (pred_voxels * target_voxels).sum()
    union = pred_voxels.sum() + target_voxels.sum() - intersection
    iou = (intersection + 1e-6) / (union + 1e-6)
    
    # Compute precision and recall
    true_positives = (pred_voxels * target_voxels).sum()
    precision = true_positives / (pred_voxels.sum() + 1e-6)
    recall = true_positives / (target_voxels.sum() + 1e-6)
    
    return {
        'iou': iou.item(),
        'precision': precision.item(),
        'recall': recall.item()
    }

In [10]:
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Грузим данные
print("\nLoading dataset...")
dataset = VoxelDataset(
    root_dir='/kaggle/input/reconstruction',
    voxel_size=32
)
print(f"Total dataset size: {len(dataset)} samples")

# Делим на обучающую и валидационные выборки
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=1,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=1,
    pin_memory=True
)

# Инициализируем модель
print("\nCreating model...")
model = EncoderDecoder3D(voxel_size=32)
model = model.to(device)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.001,
    weight_decay=1e-5
)

scheduler = CosineAnnealingLR(
    optimizer,
    T_max=100,
    eta_min=1e-6
)

print("\nInitializing trainer...")
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device
)

print("\nStarting training...")
trainer.train(num_epochs=35)

# Проверка на валидационных данных
print("\nPerforming final evaluation...")
model.eval()
total_metrics = {
    'iou': 0.0,
    'precision': 0.0,
    'recall': 0.0
}
num_batches = 0

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Evaluating"):
        images = batch['image'].to(device)
        target_voxels = batch['voxels'].to(device)
        
        pred_voxels = model(images)
        metrics = evaluate_voxels(pred_voxels, target_voxels)
        for k, v in metrics.items():
            total_metrics[k] += v
        num_batches += 1

avg_metrics = {k: v / num_batches for k, v in total_metrics.items()}

print("\nFinal Evaluation Results:")
print(f"IoU: {avg_metrics['iou']:.4f}")
print(f"Precision: {avg_metrics['precision']:.4f}")
print(f"Recall: {avg_metrics['recall']:.4f}")

print("\nSaving final model...")
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'final_metrics': avg_metrics
}, 'final_model.pth')

print("\nTraining and evaluation completed!")

Using device: cuda

Loading dataset...
Preloading voxel grids...


Loading voxel grids: 100%|████████████████████████████████████████| 196/196 [00:02<00:00, 85.92it/s]


Total dataset size: 196 samples

Creating model...

Initializing trainer...

Starting training...
Starting training for 35 epochs...
Training on device: cuda
Number of training batches: 39
Number of validation batches: 10


Validation 0: 100%|██████████| 10/10 [00:01<00:00,  5.26it/s, val_loss=0.4109, iou=0.0767, dice=0.1424]



Epoch 0:
Train Loss: 0.6153, IoU: 0.0826, Dice: 0.1521
Val Loss: 0.4156, IoU: 0.0917, Dice: 0.1674


...


Epoch 34:
Train Loss: 0.1324, IoU: 0.0826, Dice: 0.1522
Val Loss: 0.2468, IoU: 0.0917, Dice: 0.1674


Generating validation predictions: 100%|██████████| 10/10 [00:50<00:00,  5.09s/it]


Training completed!

Performing final evaluation...


Evaluating: 100%|██████████| 10/10 [00:01<00:00,  5.45it/s]



Final Evaluation Results:
IoU: 0.0917
Precision: 0.0917
Recall: 1.0000

Saving final model...

Training and evaluation completed!


In [12]:
!zip preds /kaggle/working/validation_predictions/*

  adding: kaggle/working/validation_predictions/part_0000.png_prediction.png (deflated 6%)
  adding: kaggle/working/validation_predictions/part_0021.png_prediction.png (deflated 4%)
  adding: kaggle/working/validation_predictions/part_0024.png_prediction.png (deflated 6%)
  adding: kaggle/working/validation_predictions/part_0025.png_prediction.png (deflated 7%)
  adding: kaggle/working/validation_predictions/part_0030.png_prediction.png (deflated 5%)
  adding: kaggle/working/validation_predictions/part_0035.png_prediction.png (deflated 4%)
  adding: kaggle/working/validation_predictions/part_0036.png_prediction.png (deflated 6%)
  adding: kaggle/working/validation_predictions/part_0037.png_prediction.png (deflated 5%)
  adding: kaggle/working/validation_predictions/part_0038.png_prediction.png (deflated 4%)
  adding: kaggle/working/validation_predictions/part_0045.png_prediction.png (deflated 5%)
  adding: kaggle/working/validation_predictions/part_0054.png_prediction.png (deflated 5%)